1. kaggle (https://www.kaggle.com/datasets)
2. спарсить самостоятельно, но в этом случае вам может понадобиться разметка (если у вас обучение с учителем)

Пишем простой граббер для Telegram чатов на Python https://medium.com/swlh/build-a-telegram-bot-in-go-in-9-minutes-e06ad38acef1  https://proglib.io/p/pishem-prostoy-grabber-dlya-telegram-chatov-na-python-2019-11-06

Spam Classifier in Python from scratch https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

How to deploy ML models with Telegram Bots https://medium.com/analytics-vidhya/how-to-deploy-machine-learning-models-on-android-and-ios-with-telegram-bots-a6fb16922741

In [7]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 110)
import numpy as np
import re

#from gensim.test.utils import common_texts
#from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

#from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, \
                            classification_report, precision_recall_curve, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

#import dill
import os

In [8]:
path_to_files = '../../data/spam_detection/'

ham_raw = pd.read_json(os.path.join(path_to_files, 'ham.json'))
spam_raw = pd.read_json(os.path.join(path_to_files, 'spam.json'))

# Data processing

## Обработка неспамных сообщений

In [9]:
ham_raw.tail(3)

name               type          id  \
20442  Новое Измайлово  public_supergroup  1077014744   
20443  Новое Измайлово  public_supergroup  1077014744   
20444  Новое Измайлово  public_supergroup  1077014744   

                                                                                                            messages  
20442  {'id': 84724, 'type': 'message', 'date': '2021-08-24T21:09:27', 'from': 'Juriy', 'from_id': 'user229200647...  
20443  {'id': 84727, 'type': 'message', 'date': '2021-08-24T21:57:16', 'edited': '2021-08-24T21:57:23', 'from': '...  
20444  {'id': 84728, 'type': 'message', 'date': '2021-08-24T22:10:24', 'from': 'Fidel', 'from_id': 'user429545993...

In [45]:
ham = pd.json_normalize(ham_raw['messages'])
ham.head(5)

id     type                 date           from        from_id  \
0  58320  message  2020-07-07T01:21:52       Alexandr  user362421356   
1  58321  service  2020-07-07T01:22:12            NaN            NaN   
2  58322  message  2020-07-07T01:22:32            Lia  user587895366   
3  58323  message  2020-07-07T01:22:47         Ismail  user589798156   
4  58324  message  2020-07-07T01:23:32  Maria Gaurikh  user292667992   

                                                              text   actor  \
0                                 Че за возня опять? Возле верного     NaN   
1                                                                   Ismail   
2  Что то похожее на стрельбу было и сейчас три машины полицейских     NaN   
3                                А кто-то в курсе что за стрельба?     NaN   
4                        Предупредительный был, видимо , от ментов     NaN   

        actor_id          action   members               edited  \
0            NaN             NaN       NaN                  NaN   
1  user589798156  invite_members  [Ismail]                  NaN   
2            NaN             NaN       NaN                  NaN   
3            NaN             NaN       NaN  2020-07-07T01:23:13   
4            NaN             NaN       NaN                  NaN   

  forwarded_from file thumbnail media_type mime_type  duration_seconds  width  \
0            NaN  NaN       NaN        NaN       NaN               NaN    NaN   
1            NaN  NaN       NaN        NaN       NaN               NaN    NaN   
2            NaN  NaN       NaN        NaN       NaN               NaN    NaN   
3            NaN  NaN       NaN        NaN       NaN               NaN    NaN   
4            NaN  NaN       NaN        NaN       NaN               NaN    NaN   

   height  reply_to_message_id photo contact_information.first_name  \
0     NaN                  NaN   NaN                            NaN   
1     NaN                  NaN   NaN                            NaN   
2     NaN                  NaN   NaN                            NaN   
3     NaN                  NaN   NaN                            NaN   
4     NaN                  NaN   NaN                            NaN   

  contact_information.last_name contact_information.phone_number  \
0                           NaN                              NaN   
1                           NaN                              NaN   
2                           NaN                              NaN   
3                           NaN                              NaN   
4                           NaN                              NaN   

  sticker_emoji contact_vcard  location_information.latitude  \
0           NaN           NaN                            NaN   
1           NaN           NaN                            NaN   
2           NaN           NaN                            NaN   
3           NaN           NaN                            NaN   
4           NaN           NaN                            NaN   

   location_information.longitude via_bot  message_id  
0                             NaN     NaN         NaN  
1                             NaN     NaN         NaN  
2                             NaN     NaN         NaN  
3                             NaN     NaN         NaN  
4                             NaN     NaN         NaN

In [46]:
# удалям строки сервисных сообщений (около 10-ти)
ham = ham.drop(ham[ham['type']== 'service'].index)

In [47]:
ham.head(5)

id     type                 date           from        from_id  \
0  58320  message  2020-07-07T01:21:52       Alexandr  user362421356   
2  58322  message  2020-07-07T01:22:32            Lia  user587895366   
3  58323  message  2020-07-07T01:22:47         Ismail  user589798156   
4  58324  message  2020-07-07T01:23:32  Maria Gaurikh  user292667992   
6  58326  message  2020-07-07T01:36:05  Andrey Skalny  user473275267   

                                                              text actor  \
0                                 Че за возня опять? Возле верного   NaN   
2  Что то похожее на стрельбу было и сейчас три машины полицейских   NaN   
3                                А кто-то в курсе что за стрельба?   NaN   
4                        Предупредительный был, видимо , от ментов   NaN   
6                    Всё закончилось. Разборки были с поножовщиной   NaN   

  actor_id action members               edited forwarded_from file thumbnail  \
0      NaN    NaN     NaN                  NaN            NaN  NaN       NaN   
2      NaN    NaN     NaN                  NaN            NaN  NaN       NaN   
3      NaN    NaN     NaN  2020-07-07T01:23:13            NaN  NaN       NaN   
4      NaN    NaN     NaN                  NaN            NaN  NaN       NaN   
6      NaN    NaN     NaN                  NaN            NaN  NaN       NaN   

  media_type mime_type  duration_seconds  width  height  reply_to_message_id  \
0        NaN       NaN               NaN    NaN     NaN                  NaN   
2        NaN       NaN               NaN    NaN     NaN                  NaN   
3        NaN       NaN               NaN    NaN     NaN                  NaN   
4        NaN       NaN               NaN    NaN     NaN                  NaN   
6        NaN       NaN               NaN    NaN     NaN                  NaN   

  photo contact_information.first_name contact_information.last_name  \
0   NaN                            NaN                           NaN   
2   NaN                            NaN                           NaN   
3   NaN                            NaN                           NaN   
4   NaN                            NaN                           NaN   
6   NaN                            NaN                           NaN   

  contact_information.phone_number sticker_emoji contact_vcard  \
0                              NaN           NaN           NaN   
2                              NaN           NaN           NaN   
3                              NaN           NaN           NaN   
4                              NaN           NaN           NaN   
6                              NaN           NaN           NaN   

   location_information.latitude  location_information.longitude via_bot  \
0                            NaN                             NaN     NaN   
2                            NaN                             NaN     NaN   
3                            NaN                             NaN     NaN   
4                            NaN                             NaN     NaN   
6                            NaN                             NaN     NaN   

   message_id  
0         NaN  
2         NaN  
3         NaN  
4         NaN  
6         NaN

Список признаков

In [32]:
ham.columns

Index(['id', 'type', 'date', 'from', 'from_id', 'text', 'actor', 'actor_id',
       'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
       'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
       'reply_to_message_id', 'photo', 'contact_information.first_name',
       'contact_information.last_name', 'contact_information.phone_number',
       'sticker_emoji', 'contact_vcard', 'location_information.latitude',
       'location_information.longitude', 'via_bot', 'message_id'],
      dtype='object')

Уникальные значения по некоторым признакам

In [33]:
all_cols = ['id', 'type', 'date', 'from', 'from_id', 'text', 'actor', 'actor_id',
           'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
           'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
           'reply_to_message_id', 'photo', 'contact_information.first_name',
           'contact_information.last_name', 'contact_information.phone_number',
           'sticker_emoji', 'contact_vcard', 'location_information.latitude',
           'location_information.longitude', 'via_bot', 'message_id']

some_cols = ['type', 'actor_id', 'action', 'file', 'thumbnail', 'media_type',
             'sticker_emoji', 'photo', 'mime_type', 'via_bot',
            ]

for col in some_cols:
    print(col, ham[col].unique(), '\n')

type ['message' 'service'] 

actor_id [nan 'user589798156' 'user786448865' 'user572801898' 'user1140663320'
 'user1177305831' 'user450782284'] 

action [nan 'invite_members' 'group_call' 'invite_to_group_call' 'pin_message'] 

file [nan '(File not included. Change data exporting settings to download.)'] 

thumbnail [nan '(File not included. Change data exporting settings to download.)'] 

media_type [nan 'video_file' 'sticker' 'voice_message' 'animation'] 

sticker_emoji [nan '👍' '🤔' '💩' '🥺' '😂' '🍳' '😄' '😴' '🤑' '🍿' '❗️' '🙄' '😳' '😯' '😭' '😞' '👌'
 '😱' '😌' '🤟' '🤦\u200d♂️' '😤' '😔' '🤨' '😨' '😀' '😆' '😃' '👋' '👏' '😰' '😐' '😉'
 '🤷\u200d♀️' '🇷🇸' '💬' '🤷\u200d♂️' '🤢' '😡' '🤣' '😏' '😣' '😘' '😊' '🙂' '🤦' '😬'
 '⚒' '🙅\u200d♂️' '😮' '😥' '🤩' '☺️'] 

photo [nan '(File not included. Change data exporting settings to download.)'] 

mime_type [nan 'video/mp4' 'image/jpeg' 'audio/ogg'
 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
 'image/png' 'application/pdf' 'image/gif' 'application/zi

In [49]:
ham = ham.drop(['id', 'type', 'date', 'from_id', 'actor', 'actor_id',
                'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
                'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
                'reply_to_message_id', 'photo', 'contact_information.first_name',
                'contact_information.last_name', 'contact_information.phone_number',
                'contact_vcard', 'location_information.latitude',
                'location_information.longitude', 'via_bot', 'message_id'], axis=1)
ham

from  \
0           Alexandr   
2                Lia   
3             Ismail   
4      Maria Gaurikh   
6      Andrey Skalny   
...              ...   
20440   Alexandra ▪️   
20441        Татьяна   
20442          Juriy   
20443        Дмитрий   
20444          Fidel   

                                                                                             text  \
0                                                                Че за возня опять? Возле верного   
2                                 Что то похожее на стрельбу было и сейчас три машины полицейских   
3                                                               А кто-то в курсе что за стрельба?   
4                                                       Предупредительный был, видимо , от ментов   
6                                                   Всё закончилось. Разборки были с поножовщиной   
...                                                                                           ...   
20440  на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔   
20441                                        Ну вот) значит будем скоро гулять по новому парку 👏🏻   
20442                                                                 Готов на платной основе.)))   
20443                                                         Если Дёмин Луг 6/5 то это у моря)))   
20444                                                                                               

      sticker_emoji  
0               NaN  
2               NaN  
3               NaN  
4               NaN  
6               NaN  
...             ...  
20440           NaN  
20441           NaN  
20442           NaN  
20443           NaN  
20444             😄  

[20438 rows x 3 columns]

In [52]:
ham.tail(10)

from  \
20435       Татьяна   
20436           Nik   
20437       Татьяна   
20438       Дмитрий   
20439       Татьяна   
20440  Alexandra ▪️   
20441       Татьяна   
20442         Juriy   
20443       Дмитрий   
20444         Fidel   

                                                                                                                text  \
20435                                                                                                            Пмж   
20436                                                                                            у моря надо брать )   
20437                                                                                  На море только отдыхать пока)   
20438  В сухую погоду и в выходные вполне (только его гадят несознательные отдыхающие). А так чтобы вечером после...   
20439                    Главное чтобы было с кем, а где можно думаю и придумать)) а парк может когда-то и обустроят   
20440                     на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔   
20441                                                           Ну вот) значит будем скоро гулять по новому парку 👏🏻   
20442                                                                                    Готов на платной основе.)))   
20443                                                                            Если Дёмин Луг 6/5 то это у моря)))   
20444                                                                                                                  

      sticker_emoji  
20435           NaN  
20436           NaN  
20437           NaN  
20438           NaN  
20439           NaN  
20440           NaN  
20441           NaN  
20442           NaN  
20443           NaN  
20444             😄

In [75]:
def clean_msg(msg):
    if isinstance(msg, list):
        list_len = len(msg) - 2
        for pos, elem in enumerate(msg):
            if isinstance(elem, dict) and pos ==  list_len:
                for key, val in elem.items():
                    if key == 'type' and val == 'hashtag':
                        break
                    if key == 'type' and val == 'bold':
                        break
                    if key == 'type' and val == 'mention_name':
                        break
                    if key == 'text':
                        return val
    if isinstance(msg, str):
        return msg

In [30]:
# Извлечение из json текста, , замена '' на Nan, удаление строк (сообщений) без текста
# обновление индекса после удаление строк (индекс теперь по порядку)
ham = ham_raw1['text'].apply(clean_msg).replace('', np.nan).dropna().reset_index(drop=True)
ham

0                                                                   Че за возня опять? Возле верного
1                                    Что то похожее на стрельбу было и сейчас три машины полицейских
2                                                                  А кто-то в курсе что за стрельба?
3                                                          Предупредительный был, видимо , от ментов
4                                                      Всё закончилось. Разборки были с поножовщиной
                                                    ...                                             
18042    Главное чтобы было с кем, а где можно думаю и придумать)) а парк может когда-то и обустроят
18043     на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔
18044                                           Ну вот) значит будем скоро гулять по новому парку 👏🏻
18045                                                                    Готов на платной о

In [12]:
# увеличение максимальной ширины столбца
pd.set_option('max_colwidth', 110)
ham.head(50)

0                                                                                                           Всем ✋🏻
1     Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...
2                                                                                                                 /
3                                                                             не пугайтесь, немного почистил мусор)
4                                                                                                      Всем привет)
5                                                                                                       Привет всем
6                                                                                                      Добрый вечер
7                                                                                                   Доброе утро! :)
8     Дети разные бывают - если это маленькие - то родителям обьяснил бы

## Обработка сообщений со спамом

In [58]:
spam_raw.head(5)

name             type          id  \
0  Журнал НИ  private_channel  1212843637   
1  Журнал НИ  private_channel  1212843637   
2  Журнал НИ  private_channel  1212843637   
3  Журнал НИ  private_channel  1212843637   
4  Журнал НИ  private_channel  1212843637   

                                                                                                        messages  
0  {'id': 1, 'type': 'service', 'date': '2020-07-07T20:25:57', 'actor': 'Журнал НИ', 'actor_id': 'channel1212...  
1  {'id': 4, 'type': 'message', 'date': '2020-07-07T20:27:15', 'from': 'Журнал НИ', 'from_id': 'channel121284...  
2  {'id': 5, 'type': 'message', 'date': '2020-07-07T20:31:15', 'from': 'Журнал НИ', 'from_id': 'channel121284...  
3  {'id': 6, 'type': 'message', 'date': '2020-07-07T20:34:59', 'from': 'Журнал НИ', 'from_id': 'channel121284...  
4  {'id': 7, 'type': 'message', 'date': '2020-07-07T20:35:36', 'from': 'Журнал НИ', 'from_id': 'channel121284...

In [60]:
spam = pd.json_normalize(spam_raw['messages'])
spam.tail(10)

id     type                 date actor actor_id action title  \
3088  3091  message  2021-08-22T22:40:49   NaN      NaN    NaN   NaN   
3089  3092  message  2021-08-23T23:17:05   NaN      NaN    NaN   NaN   
3090  3093  message  2021-08-24T00:33:31   NaN      NaN    NaN   NaN   
3091  3094  message  2021-08-24T00:33:43   NaN      NaN    NaN   NaN   
3092  3095  message  2021-08-24T12:07:29   NaN      NaN    NaN   NaN   
3093  3096  message  2021-08-24T13:25:44   NaN      NaN    NaN   NaN   
3094  3097  message  2021-08-24T13:57:26   NaN      NaN    NaN   NaN   
3095  3098  message  2021-08-24T19:12:21   NaN      NaN    NaN   NaN   
3096  3099  message  2021-08-24T19:12:25   NaN      NaN    NaN   NaN   
3097  3100  message  2021-08-24T21:27:48   NaN      NaN    NaN   NaN   

                                                                                                               text  \
3088  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3089  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3090  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3091  [✅ Пользователь прошедший проверку разблокирован\n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'co...   
3092  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3093  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3094  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3095  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   
3096  [✅ Пользователь прошедший проверку разблокирован\n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'co...   
3097  [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...   

           from            from_id  
3088  Журнал НИ  channel1212843637  
3089  Журнал НИ  channel1212843637  
3090  Журнал НИ  channel1212843637  
3091  Журнал НИ  channel1212843637  
3092  Журнал НИ  channel1212843637  
3093  Журнал НИ  channel1212843637  
3094  Журнал НИ  channel1212843637  
3095  Журнал НИ  channel1212843637  
3096  Журнал НИ  channel1212843637  
3097  Журнал НИ  channel1212843637

In [63]:
spam.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'title', 'text',
       'from', 'from_id'],
      dtype='object')

In [64]:
for col in ['type', 'actor', 'actor_id', 'action', 'title', 'from', 'from_id']:
    print(col, spam[col].unique())

type ['service' 'message']
actor ['Журнал НИ' nan]
actor_id ['channel1212843637' nan]
action ['create_channel' nan]
title ['Журнал НИ' nan]
from [nan 'Журнал НИ']
from_id [nan 'channel1212843637']


In [65]:
spam_1 = spam['text']
spam_1.tail(15)

3083    [✅ Пользователь прошедший проверку разблокирован\n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'co...
3084    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...
3085    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'},  , {'type': 'hashtag', 'text': '#Вернулся'}, \n\nП...
3086    [✅ Пользователь прошедший проверку разблокирован\n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'co...
3087    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...
3088    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...
3089    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...
3090    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'ty...
3091    [✅ Пользователь прошедший проверку разблокирован

In [71]:
spam_1.iloc[2007]

'Добрый день, рады представить Вам тематический чат для Дачников, здесь мы обмениваемся советами по строительству, садоводству и т.д.\n\nhttps://t.me/dacha_pro'

In [70]:
clean_msg(spam_1.iloc[2003])

In [72]:
spam_2 = spam_1.apply(clean_msg)
spam_2.tail(10)

3088    None
3089    None
3090    None
3091    None
3092    None
3093    None
3094    None
3095    None
3096    None
3097    None
Name: text, dtype: object

In [73]:
# извлечение из json текста, удаление строк (сообщений) без текста,
# обновление индекса после удаление строк (индекс теперь по порядку)
spam_3 = spam_1.apply(clean_msg).dropna().reset_index(drop=True)

In [74]:
spam_3

0                                                                                                                   
1                                                                             Канал успешно добавлен в журнал группы
2                                                                                     Охуенно надежный дом Балашихе😳
3                         🪓🪓🪓🪓🪓🪓🪓\n\nЧитайте ответ по строительству!\n\n\nhttps://dobrodel.mosreg.ru//claims/4154945
4                                                       это же бандиты:\nhttps://www.youtube.com/watch?v=ygol2LCWnHs
                                                           ...                                                      
341    Всем привет, никто не желает учить китайский язык? Я репетиторствую китайским  с 2016 года, живу на улице ...
342    Уважаемые родители ❗️❗️❗️\nХочу предложить Вам услуги няни👩&zwj;👧&zwj;👦\n\nСобираю группу деток (2-5 детей...
343                  Добрый вечер,могу вам помочь с селективной 

## Получение объединённого датасета

In [23]:
# Series -> DataFrame
ham_df = ham.to_frame('msg')
# создание нового столбца с категорией спам / не спам
ham_df['spam'] = 0

spam_df = spam.to_frame('msg')
spam_df['spam'] = 1

# конкатенация датафреймов
df = pd.concat([ham_df, spam_df], axis=0, ignore_index=True)
df

msg  \
0                                                                                                            Всем ✋🏻   
1      Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...   
2                                                                                                                  /   
3                                                                              не пугайтесь, немного почистил мусор)   
4                                                                                                       Всем привет)   
...                                                                                                              ...   
54569  Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...   
54570                                                                    https://t.me/MoscowskyPravdorub/1470?single   
54571                                                                                         Большой выбор ждём вас   
54572  Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...   
54573  Здравствуйте! Услуги Психолога!\n\nПсихолог с высшим образованием и практикой более 10 лет предлагаю услуг...   

       spam  
0         0  
1         0  
2         0  
3         0  
4         0  
...     ...  
54569     1  
54570     1  
54571     1  
54572     1  
54573     1  

[54574 rows x 2 columns]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('spam', 1), 
                                                    df['spam'], random_state=0)
#save test
X_test.to_csv("X_test_spam.csv", index=None)
y_test.to_csv("y_test_spam.csv", index=None)
#save train
X_train.to_csv("X_train_spam.csv", index=None)
y_train.to_csv("y_train_spam.csv", index=None)

In [25]:
X_train

msg
3856   Они в один месяц не учли показания (программа сбойнула, не загрузилось с сайта, не туда посмотрели) и у ва...
40894                                                             Раз в час примерно, первая в районе 7 утра уходит.
7915                                                                            Это же гениально! 😂отличная идея!)))
25371                                             Да непонятно, в доброделе отвечают — проведите ОСС и сами залейте)
40093                                                            Может одна новая, пробега мало, а вторая загаженная
...                                                                                                              ...
45891                                                                                                      Спасибо 😀
52416                                                                                            А разве заставляют?
42613                                                              Не то футбольный праздник, не то свадьба в Чечне.
43567                                                                                        А чат собаководов есть?
2732                                                                           Отправьте в добродел, отличный ракурс

[40930 rows x 1 columns]

In [26]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [27]:
features = ['msg']
target = 'spam'

In [28]:
#combine
msg = Pipeline([
                ('imputer', TextImputer('msg', '')),
                ('selector', ColumnSelector(key='msg')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('msg', msg)])

In [29]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Wall time: 723 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('msg',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='msg',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='msg')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10))]))])),
                ('classifier', LogisticRegression())])

Посмотрим, как выглядит наш pipeline

In [30]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('msg',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='msg',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='msg')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))]))])),
 ('classifier', LogisticRegression())]

In [31]:
predictions = pipeline.predict_proba(X_test)
#pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [32]:
y_test

48502    0
48785    0
40039    0
40399    0
50011    0
        ..
27058    0
16446    0
42301    0
11306    0
31998    0
Name: spam, Length: 13644, dtype: int64

In [33]:
predictions[:, 1][:]

array([0.00166743, 0.00600112, 0.00191033, ..., 0.0022781 , 0.00163441,
       0.00238243])

In [34]:
roc_auc_score(y_score=predictions[:, 1], y_true=y_test)

0.9552632018716577

In [35]:
f1_score(predictions[:, 1].round(), y_test)

0.0

In [36]:
precision_score(predictions[:, 1].round(), y_test)

0.0

In [37]:
recall_score(predictions[:, 1].round(), y_test)

C:\Users\Alex\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [38]:
df_pred = pd.concat([df, pd.Series(predictions[:, 1])], axis=1)
df_pred.columns=["msg", "spam", "prob"]
df_pred

msg  \
0                                                                                                            Всем ✋🏻   
1      Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...   
2                                                                                                                  /   
3                                                                              не пугайтесь, немного почистил мусор)   
4                                                                                                       Всем привет)   
...                                                                                                              ...   
54569  Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...   
54570                                                                    https://t.me/MoscowskyPravdorub/1470?single   
54571                                                                                         Большой выбор ждём вас   
54572  Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...   
54573  Здравствуйте! Услуги Психолога!\n\nПсихолог с высшим образованием и практикой более 10 лет предлагаю услуг...   

       spam      prob  
0         0  0.001667  
1         0  0.006001  
2         0  0.001910  
3         0  0.001192  
4         0  0.004088  
...     ...       ...  
54569     1       NaN  
54570     1       NaN  
54571     1       NaN  
54572     1       NaN  
54573     1       NaN  

[54574 rows x 3 columns]

In [39]:
df_pred.iloc[54570,2], type(df_pred.iloc[54570,2])

(nan, numpy.float64)

In [40]:
pd.set_option('display.max_rows', 200)
df_pred.loc[df_pred['prob'] > 0.1]

msg  \
3458                                                                                                       Брат окно   
4849                                                                           Воробей всегда подрузамевает)))))))))   
6068                                                                        Они по проекту района там предполагались   
7398                                                                            Желания нет продолжать эту беседу...   
8806               https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku   
9038                                                                                             Включенные габариты   
9071                                                                         Да, смартфон Android, компьютер Windows   
9257   Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...   
9632                         И кстати вот тут не за что извиняться мы ж русские , и  "вы" на Руси было оскорбительно   
9922                                                                                                       ЗАметано)   
10149                                                              Ну и ищите, пойду инвесторов в дивизию предлагать   
10625  Если брать за понятие "живёт" - ставит себе на стол тарелку с супом, то да ЖИВЁТ. А если брать возможности...   
11090                                                                       жена тоже рыба блять, молчит в  тряпочку   
13434                                                                                 Кстати, фильм очень наглядный.   

       spam      prob  
3458      0  0.165447  
4849      0  0.143628  
6068      0  0.161846  
7398      0  0.161846  
8806      0  0.161846  
9038      0  0.161846  
9071      0  0.117427  
9257      0  0.161846  
9632      0  0.166958  
9922      0  0.111680  
10149     0  0.161846  
10625     0  0.179452  
11090     0  0.161846  
13434     0  0.161846

In [41]:
pd.set_option('display.max_rows', 200)
df_pred.query('(prob > 0.15)').head(200)

msg  \
3458                                                                                                       Брат окно   
6068                                                                        Они по проекту района там предполагались   
7398                                                                            Желания нет продолжать эту беседу...   
8806               https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku   
9038                                                                                             Включенные габариты   
9257   Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...   
9632                         И кстати вот тут не за что извиняться мы ж русские , и  "вы" на Руси было оскорбительно   
10149                                                              Ну и ищите, пойду инвесторов в дивизию предлагать   
10625  Если брать за понятие "живёт" - ставит себе на стол тарелку с супом, то да ЖИВЁТ. А если брать возможности...   
11090                                                                       жена тоже рыба блять, молчит в  тряпочку   
13434                                                                                 Кстати, фильм очень наглядный.   

       spam      prob  
3458      0  0.165447  
6068      0  0.161846  
7398      0  0.161846  
8806      0  0.161846  
9038      0  0.161846  
9257      0  0.161846  
9632      0  0.166958  
10149     0  0.161846  
10625     0  0.179452  
11090     0  0.161846  
13434     0  0.161846

Сохраним модель (пайплайн)

In [42]:
with open("spam_classifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)